In [ ]:
!pip install unsloth vllm
!pip install triton==3.1.0
!pip install -U pynvml

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 1024 # Can increase for longer reasoning traces

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "benjaminzwhite/Qwen2.5-3B-Instruct_GSM8K-GRPO_LoRA-adapters",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
SAVE_NAME = "Qwen2.5-3B-Instruct"

In [ ]:
model.push_to_hub_merged(f"benjaminzwhite/{SAVE_NAME}_GSM8K-GRPO_16bit", tokenizer, save_method = "merged_16bit")

# Now try downloading full model and doing inference with it

- Here using default huggingface code, no unsloth inference stuff
- I'm copying the inference code from [https://huggingface.co/Qwen/Qwen2.5-3B-Instruct](https://huggingface.co/Qwen/Qwen2.5-3B-Instruct) as-is
- **TODO: understand the vLLM setup and why it doesn't work on local GPU machine :(**

# IMPORTANT

- I restarted kernel and only ran from below this cell, otherwise with previous stuff loaded I was getting :

`Error: 'Qwen2ForCausalLM' object has no attribute 'max_seq_length'`and didn't understand why

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "benjaminzwhite/Qwen2.5-3B-Instruct_GSM8K-GRPO_16bit"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

In [2]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

In [3]:
user_prompt = "How many r's are in strawberry?"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": user_prompt}
]

In [4]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [5]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [6]:
response

'<reasoning>\nTo find out how many \'r\'s are in "strawberry", we need to count each occurrence of the letter \'r\' in the word. The word "strawberry" contains \'r\' twice: once in the beginning and once in the middle.\n</reasoning>\n<answer>\n2\n</answer>'

## Test some difficult mathematics questions

In [7]:
def generate_answers(input_query):
    messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": input_query}
    ]
    text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [8]:
generate_answers("What is the smallest prime number greater than 50 ?")

"<reasoning>\nTo find the smallest prime number greater than 50, we can start checking from 51 onwards for primality. A prime number is a number that has no divisors other than 1 and itself. We check each number to see if it's divisible by any number other than 1 and itself.\n</reasoning>\n<answer>\n53\n</answer>"

In [9]:
generate_answers("I arrange N people in a line, then permute them at random. As N tends to infinity, what is the probability that no people end up in their original position ?")

"<reasoning>\nThe problem presented is a classic example of the derangement problem, which calculates the number of permutations where no element appears in its original position. For N people, the total number of permutations is N!. The number of derangements (denoted as !N) can be approximated using the formula: !N ≈ N! / e, where e is Euler's number (approximately 2.71828). When N approaches infinity, the probability P(N) that no one ends up in their original position is approximately equal to the limit of !(N) / N! as N → ∞, which simplifies to 1/e. Thus, the probability that no person ends up in their original position as N tends to infinity is 1/e.\n</reasoning>\n<answer>\n1/e\n</answer>"

In [11]:
generate_answers("What is the first 3-digit Catalan number ?")

"<reasoning>\nThe Catalan numbers are a sequence of natural numbers with applications in combinatorial mathematics. The nth Catalan number can be calculated using the formula: C(n) = (1/(n+1)) * binomial(2*n, n). We need to find the first 3-digit Catalan number. Let's check the first few Catalan numbers by calculating them manually or using code until we find one that is at least 100.\n</reasoning>\n<answer>\n143\n</answer>"

In [12]:
generate_answers("A given graph on 7 vertices has 20 edges. What is the smallest possible degree among all the vertices in the graph?")

"<reasoning>\nTo find the smallest possible degree among all vertices, we first need to understand the maximum number of edges a graph with 7 vertices can have without forming an isolated vertex (a vertex with no edges). This is the case for a complete graph, where each vertex is connected to every other vertex. The formula for the number of edges in a complete graph with \\(n\\) vertices is \\(\\frac{n(n-1)}{2}\\). For \\(n=7\\), this gives us \\(\\frac{7 \\cdot 6}{2} = 21\\) edges. Since our graph has 20 edges, it's missing one edge compared to the complete graph. If the degree of the vertex with the smallest degree is \\(d\\), then there are \\(7-d\\) vertices that are not connected to this vertex, and thus there would be at least one missing edge connecting these two sets of vertices. Therefore, the smallest possible degree must be 3, because if it were less than 3, we wouldn't be able to form a graph with only 20 edges out of 21 possible edges.\n</reasoning>\n<answer>\n3\n</answer

In [13]:
generate_answers("I have a 5 x 5 grid of squares, and I color each square either white or black at random. Will there always be 4 squares of the same color that form the 4 vertices of a rectangle ?")

"<reasoning>\nWe can approach this problem using the pigeonhole principle and combinatorial reasoning. Consider a 5x5 grid, which has 25 squares. If we color each square either white or black, we need to determine if it's possible to find four squares of the same color forming the vertices of a rectangle.\n\nFirstly, let's look at the structure of a rectangle formed by four squares. A rectangle in a grid can be defined by two pairs of parallel lines (one horizontal and one vertical) passing through the grid. Each pair of parallel lines will define a unique set of 4 squares forming a rectangle.\n\nNow, consider the maximum number of such rectangles we can form within a 5x5 grid. The number of ways to choose 2 rows from 5 is C(5,2) = 10, and the number of ways to choose 2 columns from 5 is also C(5,2) = 10. Therefore, the total number of rectangles in a 5x5 grid is 10 * 10 = 100.\n\nFor every 100 rectangles, there must be at least one rectangle whose four vertices are all the same color.